In [2]:
import os
import pandas as pd
import fnmatch

def read_csv_with_encoding(file, encodings):
    for encoding in encodings:
        try:
            return pd.read_csv(file, encoding=encoding)
        except UnicodeDecodeError:
            continue
    raise ValueError(f"None of the encodings worked for file: {file}")


def combine_csv_in_folder(folder_path, file_pattern):
    dataframes = []  # 创建一个空列表来收集数据帧
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if fnmatch.fnmatch(file, file_pattern):  # 使用fnmatch来进行模式匹配
                file_path = os.path.join(root, file)
                df = read_csv_with_encoding(file_path, ['utf-8', 'gb2312', 'gbk'])
                dataframes.append(df)  # 将数据帧添加到列表中
    return pd.concat(dataframes, ignore_index=True) if dataframes else pd.DataFrame()


root_folders = ['/Users/zhangwenbo/Downloads/1','/Users/zhangwenbo/Downloads/2']
file_pattern = 'ss2404.csv'
# root_folders = ['/Users/zhangwenbo/testProjects/laplace']
# file_pattern = 'ss*.csv'


# Create an empty DataFrame to store combined data
combined_csv = pd.DataFrame()

# Process each root folder and concatenate the results
for folder in root_folders:
    folder_csv = combine_csv_in_folder(folder, file_pattern)
    if not folder_csv.empty:
        combined_csv = pd.concat([combined_csv, folder_csv], ignore_index=True)

# Save the combined data to a new CSV file
combined_csv.to_csv("/Users/zhangwenbo/testProjects/laplace/taobao_combined.csv", index=False, encoding='utf-8', errors='ignore')

print("All files combined successfully!")


All files combined successfully!


In [4]:
import pandas as pd
data = pd.read_csv("/Users/zhangwenbo/testProjects/laplace/taobao_combined.csv")
code = "SHFE.ss2404."
# code = "KQ.m@SHFE.ss."

# 遍历所有列名并处理
new_columns = []
for col in data.columns:
    if col.startswith(code):
        new_columns.append(col.replace(code, ""))
    else:
        new_columns.append(col)
data.columns = new_columns
data = data.sort_values(by='datetime')
data.to_csv("future_taobao_ss2404_tick.csv",index=False)
